In [2]:
import numpy as np
import math
import gurobipy as gp
from gurobipy import GRB
from scipy.stats import poisson
from scipy.special import gamma
from scipy.integrate import trapezoid

##I forgot to change the notation form p to r here inside the code, but in the output it is r
# ==============================================
# Model Parameters
# ==============================================
scenarios = ["optimistic", "most_likely", "Pessimistic"]
alpha_dict = {"optimistic": 0.1,"most_likely": 0.15, "Pessimistic": 0.2}
prob_dict = {"optimistic": 0.2, "most_likely": 0.7, "Pessimistic": 0.1}

# System parameters
m = 50
k = 10
n_max = 13
beta = 3
mu_p = 60.83
mu_q = 30.42
C_LRU = 50000
c_u = 3000
c_v = 4500
c_h = 10000
c_p = 480000
c_q = 640000
phi1 = 0.1295
phi2 = 0.2310
A_min = 0.9
max_p = 10
max_q = 10
max_y = 40
t_s = 0.000913
M = 10.0  # Upper bound for t_ATT[s]

# Precompute MTBF
#T_bar = {s: (1 / alpha_dict[s]) * gamma(1 + 1 / beta) for s in scenarios} #I don't need it as I set the tau bounds manually 

# Shared tau parameters across all scenarios
tau_min =2 #Approximately given the lower limit and upper limit
tau_max = 9 #For computational flexibility 
num_tau_points = 10
tau_points = np.linspace(tau_min, tau_max, num_tau_points)

# Precompute reliability and integrals using shared tau points
R_tau_precomp = {s: [math.exp(-(alpha_dict[s] * t) ** beta) for t in tau_points] for s in scenarios}
F_tau_precomp = {s: [1 - r for r in R_tau_precomp[s]] for s in scenarios}
integral_precomp = {s: [] for s in scenarios}

for s in scenarios:
    for t in tau_points:
        time_grid = np.linspace(0, t, 50)
        R_vals = np.array([math.exp(-(alpha_dict[s] * ti) ** beta) for ti in time_grid])
        integral_val = trapezoid(R_vals, time_grid)
        integral_precomp[s].append(integral_val)

# System reliability precomputation (k-out-of-n)
reliability_precomp = {n: [] for n in range(k, n_max + 1)}
R_comp_vals = np.linspace(0, 1, 50)  # Component reliability range
for n in range(k, n_max + 1):
    reliability_precomp[n] = [
        sum(math.comb(n, j) * (R ** j) * ((1 - R) ** (n - j))
        for j in range(k, n + 1))
        for R in R_comp_vals
    ]

# Precompute binomial availability
A_vals = np.linspace(0.1, 1.0, 50)
binom_precomp = {n: [] for n in range(k, n_max + 1)}
for n in range(k, n_max + 1):
    for A in A_vals:
        val = sum(math.comb(n, j) * (A ** j) * ((1 - A) ** (n - j)) for j in range(k, n + 1))
        binom_precomp[n].append(val)

# Precompute Erlang C and B
erlang_points = np.linspace(0.1, 0.999, 10)
erlang_c_precomp = {p: [] for p in range(1, max_p + 1)}
erlang_b_precomp = {q: [] for q in range(1, max_q + 1)}
for p in range(1, max_p + 1):
    for rho in erlang_points:
        a = p * rho
        numerator = (a ** p) / (math.factorial(p) * (1 - rho))
        denominator = sum((a ** j) / math.factorial(j) for j in range(p)) + numerator
        erlang_c_precomp[p].append(numerator / denominator if denominator != 0 else 0)
for q in range(1, max_q + 1):
    for rho in erlang_points:
        numerator = (q * rho) ** q / math.factorial(q)
        denominator = sum((q * rho) ** j / math.factorial(j) for j in range(q + 1))
        erlang_b_precomp[q].append(numerator / denominator if denominator != 0 else 0)

# Precompute Poisson CDF
x_vals_poisson = np.linspace(0, 30, 100)
poisson_precomp = {y: [1 - poisson.cdf(y, x) for x in x_vals_poisson] for y in range(max_y + 1)}

# ==============================================
# Model Setup
# ==============================================
model = gp.Model("SimplifiedMaintenanceOptimization")
model.setParam('OutputFlag', 1)
model.setParam('NonConvex', 2)

# Decision variables (first stage)
x = model.addVar(vtype=GRB.INTEGER, lb=0, ub=n_max - k, name="x")
tau = model.addVar(vtype=GRB.CONTINUOUS, lb=tau_min, ub=tau_max, name="tau")

# Scenario-specific variables (second stage)
p = model.addVars(scenarios, vtype=GRB.INTEGER, lb=1, ub=max_p, name="p")
q = model.addVars(scenarios, vtype=GRB.INTEGER, lb=1, ub=max_q, name="q")
y = model.addVars(scenarios, vtype=GRB.INTEGER, lb=0, ub=max_y, name="y")

# Auxiliary variables - now using single set of tau_bin variables
tau_bin = model.addVars(num_tau_points, vtype=GRB.BINARY, name="tau_bin")
x_bin = model.addVars(n_max - k + 1, vtype=GRB.BINARY, name="x_bin")

#Reliability 
R_system = model.addVars(scenarios, lb=0, ub=1, name="R_system")
expected_reliability = model.addVar(lb=0, ub=1, name="expected_reliability")

# Other variables remain scenario-specific
R_tau = model.addVars(scenarios, lb=0, ub=1, name="R_tau")
F_tau = model.addVars(scenarios, lb=0, ub=1, name="F_tau")
integral = model.addVars(scenarios, lb=0, name="integral")
lambda_p = model.addVars(scenarios, lb=0, name="lambda_F_p")
lambda_q = model.addVars(scenarios, lb=0, name="lambda_F_q")
lambda_F_p = model.addVars(scenarios, lb=0, name="lambda_F_p")
lambda_F_q = model.addVars(scenarios, lb=0, name="lambda_F_q")
rho_p = model.addVars(scenarios, lb=0, ub=0.999, name="rho_p")
rho_q = model.addVars(scenarios, lb=0, ub=0.999, name="rho_q")
C_p = model.addVars(scenarios, lb=0, ub=1, name="C_p")
B_q = model.addVars(scenarios, lb=0, ub=1, name="B_q")
t_p = model.addVars(scenarios, lb=0, name="t_p")
t_q = model.addVars(scenarios, lb=0, name="t_q")
t_ATT = model.addVars(scenarios, lb=0, ub=M, name="t_ATT")
mu_m = model.addVars(scenarios, lb=0, name="mu_m")
Pr_O_y = model.addVars(scenarios, lb=0, ub=1, name="Pr_O_y")
A = model.addVars(scenarios, lb=0, ub=1, name="A")
A_R = model.addVars(scenarios, lb=0, ub=1, name="A_R")
cost_s = model.addVars(scenarios, lb=0, name="cost_s")
total_cost = model.addVar(lb=0, name="total_cost")
w = model.addVars(scenarios, lb=0, ub=M, name="w")

# ==============================================
# Constraints
# ==============================================
# Tau selection constraints 
model.addConstr(gp.quicksum(tau_bin[i] for i in range(num_tau_points)) == 1, name="tau_bin_sum")
model.addConstr(tau == gp.quicksum(tau_points[i] * tau_bin[i] for i in range(num_tau_points)), name="tau_select")

for s in scenarios:
    # Link precomputed values using shared tau_bin variables
    model.addConstr(R_tau[s] == gp.quicksum(R_tau_precomp[s][i] * tau_bin[i] for i in range(num_tau_points)), name=f"R_tau_{s}")
    model.addConstr(F_tau[s] == gp.quicksum(F_tau_precomp[s][i] * tau_bin[i] for i in range(num_tau_points)), name=f"F_tau_{s}")
    model.addConstr(integral[s] == gp.quicksum(integral_precomp[s][i] * tau_bin[i] for i in range(num_tau_points)), name=f"integral_{s}")
    
    # Tau bounds (using shared bounds)
    model.addConstr(tau >= tau_min, name=f"tau_min_{s}")
    model.addConstr(tau <= tau_max, name=f"tau_max_{s}")
    
    # Select x
    model.addConstr(gp.quicksum(x_bin[x_val] for x_val in range(n_max - k + 1)) == 1, name="x_bin_sum")
    model.addConstr(x == gp.quicksum(x_val * x_bin[x_val] for x_val in range(n_max - k + 1)), name="x_select")
    
    # spare part demand rates
    model.addConstr(lambda_p[s] == R_tau[s] / integral[s], name=f"lambda_p_{s}")
    model.addConstr(lambda_q[s] == F_tau[s] / integral[s], name=f"lambda_q_{s}")
    model.addConstr(lambda_F_p[s] == m * (k + x) * lambda_p[s], name=f"lambda_F_p_{s}")
    model.addConstr(lambda_F_q[s] == m * (k + x) * lambda_q[s], name=f"lambda_F_q_{s}")
    
    # Queue stability
    model.addConstr(rho_p[s] == lambda_F_p[s] / (mu_p * p[s]), name=f"rho_p_def_{s}")
    model.addConstr(rho_q[s] == lambda_F_q[s] / (mu_q * q[s]), name=f"rho_q_def_{s}")
    
    # Erlang C and B via PWL with binary selection
    p_bin = model.addVars(range(1, max_p + 1), vtype=GRB.BINARY, name=f"p_bin_{s}")
    q_bin = model.addVars(range(1, max_q + 1), vtype=GRB.BINARY, name=f"q_bin_{s}")
    model.addConstr(gp.quicksum(p_bin[p_val] for p_val in range(1, max_p + 1)) == 1, name=f"p_bin_sum_{s}")
    model.addConstr(gp.quicksum(q_bin[q_val] for q_val in range(1, max_q + 1)) == 1, name=f"q_bin_sum_{s}")
    model.addConstr(p[s] == gp.quicksum(p_val * p_bin[p_val] for p_val in range(1, max_p + 1)), name=f"p_select_{s}")
    model.addConstr(q[s] == gp.quicksum(q_val * q_bin[q_val] for q_val in range(1, max_q + 1)), name=f"q_select_{s}")
    for p_val in range(1, max_p + 1):
        temp_C_p = model.addVar(lb=0, ub=1, name=f"temp_C_p_{s}_{p_val}")
        model.addGenConstrPWL(rho_p[s], temp_C_p, erlang_points, erlang_c_precomp[p_val], name=f"erlang_C_pwl_{s}_{p_val}")
        model.addGenConstrIndicator(p_bin[p_val], True, C_p[s] == temp_C_p, name=f"C_p_select_{s}_{p_val}")
    for q_val in range(1, max_q + 1):
        temp_B_q = model.addVar(lb=0, ub=1, name=f"temp_B_q_{s}_{q_val}")
        model.addGenConstrPWL(rho_q[s], temp_B_q, erlang_points, erlang_b_precomp[q_val], name=f"erlang_B_pwl_{s}_{q_val}")
        model.addGenConstrIndicator(q_bin[q_val], True, B_q[s] == temp_B_q, name=f"B_q_select_{s}_{q_val}")
    
    # Waiting times
    model.addConstr(t_p[s] == (C_p[s] / (p[s] * mu_p - lambda_F_p[s])) + (1 / mu_p), name=f"t_p_{s}")
    model.addConstr(t_q[s] == (B_q[s] / (q[s] * mu_q - lambda_F_q[s])) + (1 / mu_q), name=f"t_q_{s}")
    
    # Turnaround time
    model.addConstr(t_ATT[s] == F_tau[s] * t_q[s] + R_tau[s] * t_p[s], name=f"t_ATT_{s}")
    
    # Demand rate
    model.addConstr(mu_m[s] == (m / integral[s]) * (k + x) * t_ATT[s], name=f"mu_m_{s}")
    
    # Stockout probability
    y_bin = model.addVars(range(max_y + 1), vtype=GRB.BINARY, name=f"y_bin_{s}")
    model.addConstr(gp.quicksum(y_bin[y_val] for y_val in range(max_y + 1)) == 1, name=f"y_bin_sum_{s}")
    model.addConstr(y[s] == gp.quicksum(y_val * y_bin[y_val] for y_val in range(max_y + 1)), name=f"y_select_{s}")
    for y_val in range(max_y + 1):
        temp_pr = model.addVar(lb=0, ub=1, name=f"temp_pr_{s}_{y_val}")
        model.addGenConstrPWL(mu_m[s], temp_pr, x_vals_poisson, poisson_precomp[y_val], name=f"poisson_pwl_{s}_{y_val}")
        model.addGenConstrIndicator(y_bin[y_val], True, Pr_O_y[s] == temp_pr, name=f"poisson_select_{s}_{y_val}")
    
    # Component availability
    z = model.addVar(lb=0, name=f"z_{s}")  # Auxiliary for linearizing A[s]
    # McCormick envelope for w[s] = t_ATT[s] * Pr_O_y[s]
    model.addConstr(w[s] >= 0, name=f"w_nonneg_{s}")
    model.addConstr(w[s] >= t_ATT[s] - M * (1 - Pr_O_y[s]), name=f"w_lb_{s}")
    model.addConstr(w[s] <= t_ATT[s], name=f"w_ub1_{s}")
    model.addConstr(w[s] <= M * Pr_O_y[s], name=f"w_ub2_{s}")
    # Linearized constraint
    model.addConstr(z * (integral[s] + t_s + w[s]) == 1, name=f"z_def_{s}")
    model.addConstr(A[s] == integral[s] * z, name=f"A_{s}")
    
    # System availability
    for x_val in range(n_max - k + 1):
        n = k + x_val
        temp_A_R = model.addVar(lb=0, ub=1, name=f"temp_A_R_{s}_{x_val}")
        model.addGenConstrPWL(A[s], temp_A_R, A_vals, binom_precomp[n], name=f"A_R_pwl_{s}_{x_val}")
        model.addGenConstrIndicator(x_bin[x_val], True, A_R[s] == temp_A_R, name=f"A_R_select_{s}_{x_val}")
    
    # Cost
    model.addConstr(
        cost_s[s] == (k + x) * ((lambda_p[s] * c_u + (lambda_q[s]) * c_v)) +
                     (1 / m) * (y[s] * ((phi2 * C_LRU) + c_h) + (p[s] * c_p + q[s] * c_q)),
        name=f"cost_s_{s}"
    )

# System reliability PWL approximation
for s in scenarios:
    for x_val in range(n_max - k + 1):
        n = k + x_val
        temp_R_sys = model.addVar(lb=0, ub=1, name=f"temp_R_sys_{s}_{n}")
        model.addGenConstrPWL(R_tau[s], temp_R_sys, 
                            R_comp_vals, reliability_precomp[n],
                            name=f"R_sys_pwl_{s}_{n}")
        model.addGenConstrIndicator(x_bin[x_val], True,
                                  R_system[s] == temp_R_sys,
                                  name=f"R_sys_select_{s}_{n}")

# Expected reliability across scenarios
model.addConstr(expected_reliability == gp.quicksum(prob_dict[s] * R_system[s] 
                for s in scenarios), name="expected_reliability")

# Total Cost (First stage cost + Expected second stage cost)
model.addConstr(
    total_cost == ((k + x) * phi1 * C_LRU) + gp.quicksum(prob_dict[s] * cost_s[s] for s in scenarios),
    name="total_cost"
)

# Availability constraints
for s in scenarios:
    model.addConstr(A_R[s] >= A_min, name=f"min_availability_{s}")

# Objective: Minimize cost and Maximize Reliability 
w1=1
w2=1000000
model.setObjective(w1*total_cost-w2*expected_reliability, GRB.MINIMIZE)

#model.addConstr(expected_reliability >= 0.8, name="min_reliability")
# ==============================================
# Solve
# ==============================================
model.optimize()

if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Total expected cost: {total_cost.X:.2f}")
    print(f"x = {x.X:.0f}")
    print(f"tau = {tau.X:.4f}")
    print(f"\nExpected System Reliability: {expected_reliability.X:.4f}")
    for s in scenarios:
        print(f"\nScenario {s}:")
        print(f"  r = {p[s].X:.0f}, q = {q[s].X:.0f}, y = {y[s].X:.0f}")
        print(f"  Stockout Prob = {Pr_O_y[s].X:.4f}")
        print(f"  Availability = {A_R[s].X:.6f}")
        print(f"  {s} scenario reliability: {R_system[s].X:.4f}")
        print(f"  Integral = {integral[s].X:.6f}")
        print(f"  Spares demand for Fleet renewal = {lambda_F_p[s].X:.6f}")
        print(f"  Spares demand for Fleet repair = {lambda_F_q[s].X:.6f}")
else:
    print("No optimal solution found")
    if model.status == GRB.INFEASIBLE:
        model.computeIIS()
        model.write("model.mps")
        print("Infeasible model. Check 'model.mps' for conflicts.")


Set parameter OutputFlag to value 1
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7300HQ CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
NonConvex  2

Optimize a model with 58 rows, 483 columns and 558 nonzeros
Model fingerprint: 0xe608dd4a
Model has 18 quadratic constraints
Model has 414 simple general constraints
  207 INDICATOR, 207 PWL
Model has 21 general nonlinear constraints (24 nonlinear terms)
Variable types: 276 continuous, 207 integer (197 binary)
Coefficient statistics:
  Matrix range     [3e-03, 6e+03]
  QMatrix range    [1e+00, 5e+03]
  QLMatrix range   [9e-04, 5e+04]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 4e+01]
  RHS range        [9e-01, 6e+04]
  QRHS range       [1e+00, 1e+00]
  PWLCon x range   [2e-02, 3e+01]
  PWLCon y range   [0e+00, 1e+00]
  G